In [1]:
import time, joblib, os
import onnxruntime as ort
import numpy as np
import pandas as pd

from utils.dataset import Dataset
from tqdm.auto import tqdm

In [2]:
data_root_path = '.' + os.sep + 'data'
data_file_name_list = os.listdir(data_root_path)
data_file_name_list

['safe-boom-40-swing-180-load-40-.csv',
 'safe-boom-50-swing-180-load-50-.csv',
 'safe-boom-60-swing-180-load-60-.csv',
 'safe-boom-70-swing-180-load-100-.csv',
 'safe-boom-80-swing-180-load-120-.csv',
 'unsafe-swing-0-load-70-.csv',
 'unsafe-swing-0-load-90-.csv',
 'unsafe-swing-135-load-50-.csv',
 'unsafe-swing-135-load-70-.csv',
 'unsafe-swing-180-load-70-.csv',
 'unsafe-swing-180-load-90-.csv',
 'unsafe-swing-45-load-50-.csv',
 'unsafe-swing-45-load-70-.csv',
 'unsafe-swing-90-load-70-.csv',
 'unsafe-swing-90-load-90-.csv']

In [3]:
dataset_inst = Dataset(file_path_list=data_file_name_list)

  0%|          | 0/15 [00:00<?, ?it/s]

In [4]:
raw_dataset = dataset_inst.get_raw_dataset()

In [5]:
raw_dataset_fix = raw_dataset.copy()

raw_dataset_fix = raw_dataset_fix[raw_dataset_fix['Time(sec)'] > 2.0]

for target_name in dataset_inst.get_data_target_names():
    raw_dataset_fix = raw_dataset_fix[raw_dataset_fix[target_name] > 1000]
    
raw_dataset_fix['Boom_Angle(deg)'] = raw_dataset_fix['Boom_Angle(deg)']+70

raw_dataset_fix.reset_index(inplace=True, drop=True)

In [6]:
len_data = raw_dataset_fix.shape[0]
n_of_interval = int(len_data/(len_data*0.2))

In [7]:
val_data = raw_dataset_fix[::n_of_interval]
val_data

,Time(sec),Actual_Load_Left_1(N),Actual_Load_Left_2(N),Actual_Load_Left_3(N),Actual_Load_Left_4(N),Actual_Load_Left_5(N),Actual_Load_Right_1(N),Actual_Load_Right_2(N),Actual_Load_Right_3(N),Actual_Load_Right_4(N),Actual_Load_Right_5(N),Boom_Angle(deg),Swing_Angle(deg),Load(Ton)
0,2.000253,170062.47,180680.23,191420.21,202263.13,213189.69,168664.7200,179283.9400,190028.3600,200878.5800,211815.210,66.879271,1.560000e-30,40.0
5,2.050253,170483.87,180907.70,191448.84,202089.07,212810.14,169022.0200,179447.1800,189992.6500,200640.0800,211371.150,66.733915,9.370000e-31,40.0
10,2.100253,171219.98,181343.90,191577.62,201904.46,212307.76,169498.5900,179622.9100,189860.3000,200194.0100,210607.250,66.585904,9.870000e-30,40.0
15,2.150253,172178.57,181932.53,191787.34,201728.12,211740.01,170040.2700,179793.3400,189650.9900,199598.2500,209620.160,66.435284,3.480000e-30,40.0
20,2.200253,173074.62,182509.08,192036.91,201644.70,211319.02,170443.4100,179875.6600,189405.5600,199019.5700,208704.160,66.282099,-4.470000e-31,40.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
60850,29.920253,462051.33,464014.70,465933.24,467806.33,469633.33,8792.9026,11045.4470,13535.4500,16257.2530,19205.198,55.358831,9.000000e+01,90.0
60855,29.970253,463830.77,466038.10,468185.27,470271.32,472295.30,6649.9670,8816.9263,11268.6360,13997.9220,16997.608,55.133860,9.000000e+01,90.0
60860,30.020253,465449.65,467946.52,470361.77,472693.91,474941.47,4647.1116,6681.1238,9061.7776,11779.7840,14825.853,54.908862,9.000000e+01,90.0
60865,30.070253,466684.16,469636.63,472468.95,475178.49,477762.68,2754.5771,4601.4865,6896.1467,9625.4635,12776.343,54.683882,9.000000e+01,90.0


In [8]:
model_feature_names = ['f10', 'f11', 'f12']
model_target_names = ['f20', 'f21', 'f22', 'f23', 'f24', 'f25', 'f26', 'f27', 'f28', 'f29']

In [9]:
val_data_for_model = val_data.rename(columns=dict(zip(dataset_inst.get_data_feature_names()+dataset_inst.get_data_target_names(), model_feature_names+model_target_names)))
val_data_for_model

,Time(sec),f20,f21,f22,f23,f24,f25,f26,f27,f28,f29,f10,f11,f12
0,2.000253,170062.47,180680.23,191420.21,202263.13,213189.69,168664.7200,179283.9400,190028.3600,200878.5800,211815.210,66.879271,1.560000e-30,40.0
5,2.050253,170483.87,180907.70,191448.84,202089.07,212810.14,169022.0200,179447.1800,189992.6500,200640.0800,211371.150,66.733915,9.370000e-31,40.0
10,2.100253,171219.98,181343.90,191577.62,201904.46,212307.76,169498.5900,179622.9100,189860.3000,200194.0100,210607.250,66.585904,9.870000e-30,40.0
15,2.150253,172178.57,181932.53,191787.34,201728.12,211740.01,170040.2700,179793.3400,189650.9900,199598.2500,209620.160,66.435284,3.480000e-30,40.0
20,2.200253,173074.62,182509.08,192036.91,201644.70,211319.02,170443.4100,179875.6600,189405.5600,199019.5700,208704.160,66.282099,-4.470000e-31,40.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
60850,29.920253,462051.33,464014.70,465933.24,467806.33,469633.33,8792.9026,11045.4470,13535.4500,16257.2530,19205.198,55.358831,9.000000e+01,90.0
60855,29.970253,463830.77,466038.10,468185.27,470271.32,472295.30,6649.9670,8816.9263,11268.6360,13997.9220,16997.608,55.133860,9.000000e+01,90.0
60860,30.020253,465449.65,467946.52,470361.77,472693.91,474941.47,4647.1116,6681.1238,9061.7776,11779.7840,14825.853,54.908862,9.000000e+01,90.0
60865,30.070253,466684.16,469636.63,472468.95,475178.49,477762.68,2754.5771,4601.4865,6896.1467,9625.4635,12776.343,54.683882,9.000000e+01,90.0


In [10]:
rf_model = joblib.load('rf_model.joblib')

In [11]:
val_input = val_data_for_model[model_feature_names].values

In [12]:
rf_elapsed_time = []

for val_val in tqdm(val_input):
    input_pd = pd.DataFrame(val_val.reshape(1, -1), columns=model_feature_names)
    t0 = time.time()
    rf_pred = rf_model.predict(input_pd)
    rf_elapsed_time.append(time.time() - t0)

  0%|          | 0/12175 [00:00<?, ?it/s]

In [13]:
sum(rf_elapsed_time)

214.590918302536

In [14]:
onnx_model = ort.InferenceSession('rf_model.onnx')
input_name = onnx_model.get_inputs()[0].name
output_name = onnx_model.get_outputs()[0].name

In [15]:
input_data = val_data_for_model[model_feature_names].values.astype(np.float32)

In [16]:
onnx_elapsed_time = []

for input_val in tqdm(input_data):
    t0 =time.time()
    onnx_pred = onnx_model.run(output_names=None, input_feed={'float_input': input_val.reshape(1, -1)})[0][0]
    onnx_elapsed_time.append(time.time() - t0)

  0%|          | 0/12175 [00:00<?, ?it/s]

In [17]:
sum(onnx_elapsed_time)

2.999427080154419

In [21]:
(1-(abs(rf_pred-onnx_pred)/rf_pred))*100

array([[99.9999909 , 99.99999809, 99.99999931, 99.99998931, 99.99999151,
        99.99999918, 99.99998329, 99.99999073, 99.99998858, 99.99999327]])

In [19]:
rf_pred

array([[467142.77   , 470595.241  , 473878.403  , 476987.949  ,
        479919.572  ,   1565.76195,   3210.28081,   5421.27394,
          8180.6768 ,  11470.4246 ]])

In [20]:
onnx_pred

array([467142.8   , 470595.25  , 473878.4   , 476988.    , 479919.53  ,
         1565.762 ,   3210.2803,   5421.2734,   8180.6777,  11470.424 ],
      dtype=float32)